## Import Dataset

In [16]:
import pandas as pd

In [17]:
root = "/home/sihartist/Desktop/"

In [18]:
# Load the credit card transaction data
df = pd.read_csv('/home/sihartist/Desktop/fraud-detection/dataset/data_balanced.csv')
df = df.iloc[:,1:]

df.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER,isFlaggedFraud,isFraud
0,139,265803.35,0.00,0.00,751669.39,1017472.74,0,1,0,0,0,0,0
1,589,11278.28,0.00,0.00,220970.84,232249.12,0,1,0,0,0,0,0
2,284,16363.06,35966.16,19603.10,0.00,0.00,0,0,0,1,0,0,0
3,37,107223.31,20752629.71,20859853.02,804594.62,571711.22,1,0,0,0,0,0,0
4,287,3678.25,0.00,0.00,0.00,0.00,0,0,0,1,0,0,0


## Prepare Training And Testing Data

In [19]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [20]:
# Split the data into training and test sets
X = df.drop('isFraud', axis=1).values
y = df['isFraud'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the data for input into the CNN
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]))
print(X_train.shape)


(13140, 12)


## Build & Train Model

In [21]:
# Build the deep CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(32, 2, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.MaxPool1D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid', name='isFraud')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
411/411 [==============================] - 1s 2ms/step - loss: 0.3030 - accuracy: 0.8689
Epoch 2/10
411/411 [==============================] - 1s 2ms/step - loss: 0.2127 - accuracy: 0.9024
Epoch 3/10
411/411 [==============================] - 1s 2ms/step - loss: 0.1902 - accuracy: 0.9148
Epoch 4/10
411/411 [==============================] - 1s 2ms/step - loss: 0.1771 - accuracy: 0.9234
Epoch 5/10
411/411 [==============================] - 1s 2ms/step - loss: 0.1709 - accuracy: 0.9253
Epoch 6/10
411/411 [==============================] - 1s 2ms/step - loss: 0.1640 - accuracy: 0.9290
Epoch 7/10
411/411 [==============================] - 1s 2ms/step - loss: 0.1602 - accuracy: 0.9296
Epoch 8/10
411/411 [==============================] - 1s 2ms/step - loss: 0.1558 - accuracy: 0.9307
Epoch 9/10
411/411 [==============================] - 1s 2ms/step - loss: 0.1560 - accuracy: 0.9347
Epoch 10/10
411/411 [==============================] - 1s 2ms/step - loss: 0.1489 - accuracy: 0.9346

## Evaluate Model

In [22]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test Accuracy:', test_acc)

103/103 [==============================] - 0s 1ms/step - loss: 0.1443 - accuracy: 0.9394
Test Accuracy: 0.9394400715827942


## Save CNN Model To HDF5 Format

In [23]:
model.save("models/cnn.h5")

## Save CNN Model To ONNX Format

In [28]:
import tensorflow as tf
import tf2onnx
import onnx

loaded_model = tf.keras.models.load_model('models/cnn.h5')
onnx_model, _ = tf2onnx.convert.from_keras(loaded_model)
onnx.save(onnx_model, "models/cnn.onnx")
onnx.save(onnx_model, root + "SpringAPI/src/main/resources/models/cnn.onnx")


2023-03-01 13:41:45.687405: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-01 13:41:45.687555: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-03-01 13:41:45.687637: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-03-01 13:41:45.688236: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-01 13:41:45.688370: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gp

## Load Model And Test A Prediction

In [25]:
test=[139, 265803.35, 0.00, 0.00, 751669.39, 1017472.74, 0, 1, 0, 0, 0 ,0]
test1=[319, 1033878.67, 1033878.67, 0.0, 0.0, 0.0, 0, 0, 0, 0, 1, 0]

to_predict1 = np.array(test).reshape(1,12)
to_predict2 = np.array(test1).reshape(1,12)

loaded_model = tf.keras.models.load_model("models/cnn.h5")
result = loaded_model.predict(to_predict1)
print(result)

1/1 [==============================] - 0s 48ms/step
[[0.]]
